In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install phonetics

  Preparing metadata (setup.py) ... done
  Created wheel for phonetics: filename=phonetics-1.0.5-py2.py3-none-any.whl size=8696 sha256=8da78edf78cb2fe37c00f41518d26ff1583469f33f63a4e7b4da367d21990b2e
  Stored in directory: /root/.cache/pip/wheels/b7/1e/82/80a78c7d1ad7fc6e0af1b4d9009360b251c0e50fe59f046edb
Successfully built phonetics
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import re
from rapidfuzz import process, fuzz
from fastapi import FastAPI
import phonetics
import uvicorn
import nest_asyncio
import torch

In [8]:
df1 = pd.read_csv("/kaggle/input/az-medicine-dataset-of-india/A_Z_medicines_dataset_of_India.csv")
df2 = pd.read_csv("/kaggle/input/250k-medicines-usage-side-effects-and-substitutes/medicine_dataset.csv", 
                  dtype=str,  # Treat all columns as strings to avoid mixed-type issues
                  low_memory=False)

In [9]:
print(len(df1))
print(len(df2))

253973
248218


In [65]:
df1.head()

,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,False,Sanofi India Ltd,allopathy,strip of 15 tablets,Pheniramine (25mg),NaN


In [66]:
df2.head()

,id,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,...,sideEffect41,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,...,NaN,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,...,NaN,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN,NaN,NaN,Diphenylmethane Derivative,No,RESPIRATORY,H1 Antihistaminics (second Generation)
4,5,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,...,NaN,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation)


In [68]:
df1_selected = df1[["id", "name", "price(₹)", "manufacturer_name", "pack_size_label", 
                     "short_composition1", "short_composition2", "Is_discontinued"]].copy()

df2_selected = df2[["id", "name", "substitute0", "substitute1", "substitute2", 
                     "substitute3", "substitute4", "sideEffect0", "sideEffect1", "sideEffect2", "sideEffect3", "sideEffect4", "use0", "use1", "use2", "use3","use4", "Therapeutic Class", "Action Class"]].copy()

In [69]:
df1_selected["id"] = df1_selected["id"].astype(str).str.zfill(6)  # Adjust `6` based on your IDs
df2_selected["id"] = df2_selected["id"].astype(str).str.zfill(6)

In [70]:
df1_selected["name"] = df1_selected["name"].str.lower().str.strip()
df2_selected["name"] = df2_selected["name"].str.lower().str.strip()

In [71]:
print(len(df1_selected))
print(len(df2_selected))

253973
248218


In [72]:
print(f" Unique IDs in df1: {df1_selected['id'].nunique()}")
print(f" Unique IDs in df2: {df2_selected['id'].nunique()}")

 Unique IDs in df1: 253973
 Unique IDs in df2: 248218


In [73]:
common_ids = set(df1_selected["id"]).intersection(set(df2_selected["id"]))
common_names = set(df1_selected["name"]).intersection(set(df2_selected["name"]))

print(f" Common IDs: {len(common_ids)}")
print(f" Common Medicine Names: {len(common_names)}")

 Common IDs: 248218
 Common Medicine Names: 221415


In [74]:
print("Unique Medicine Names in df1:", df1_selected["name"].nunique())
print("Unique Medicine Names in df2:", df2_selected["name"].nunique())
print("Total Rows in df1:", len(df1_selected))
print("Total Rows in df2:", len(df2_selected))

Unique Medicine Names in df1: 249345
Unique Medicine Names in df2: 222825
Total Rows in df1: 253973
Total Rows in df2: 248218


In [75]:
df1_selected = df1_selected.drop_duplicates(subset="name")
df2_selected = df2_selected.drop_duplicates(subset="name")

In [76]:
df_merged = pd.merge(df1_selected, df2_selected, on="name", how="inner")

print(f"Total Medicines After Name-Based Merge: {len(df_merged)}")

Total Medicines After Name-Based Merge: 221415


In [77]:
# df_merged = pd.merge(df1_selected, df2_selected, on=["id", "name"], how="inner")
# print(f" Total Medicines in Dataset: {len(df)}")

In [78]:
df_merged["Quantity Available"] = np.random.randint(0,20, size=len(df_merged))

In [79]:
df_merged.head()

,id_x,name,price(₹),manufacturer_name,pack_size_label,short_composition1,short_composition2,Is_discontinued,id_y,substitute0,...,sideEffect3,sideEffect4,use0,use1,use2,use3,use4,Therapeutic Class,Action Class,Quantity Available
0,000001,augmentin 625 duo tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg),False,000001,Penciclav 500 mg/125 mg Tablet,...,NaN,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,ANTI INFECTIVES,NaN,19
1,000002,azithral 500 tablet,132.36,Alembic Pharmaceuticals Ltd,strip of 5 tablets,Azithromycin (500mg),NaN,False,000002,Zithrocare 500mg Tablet,...,Diarrhea,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,ANTI INFECTIVES,Macrolides,0
2,000003,ascoril ls syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml),False,000003,Solvin LS Syrup,...,Upset stomach,Stomach pain,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,RESPIRATORY,NaN,12
3,000004,allegra 120mg tablet,218.81,Sanofi India Ltd,strip of 10 tablets,Fexofenadine (120mg),NaN,False,000004,Lcfex Tablet,...,Nausea,NaN,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,NaN,NaN,NaN,RESPIRATORY,H1 Antihistaminics (second Generation),6
4,000005,avil 25 tablet,10.96,Sanofi India Ltd,strip of 15 tablets,Pheniramine (25mg),NaN,False,000005,Eralet 25mg Tablet,...,NaN,NaN,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,RESPIRATORY,H1 Antihistaminics (First Generation),12


In [80]:
print(f" Total Medicines in Dataset: {len(df_merged)}")

 Total Medicines in Dataset: 221415


In [81]:
print(df_merged.isnull().sum())  # Check missing values

id_x                       0
name                       0
price(₹)                   0
manufacturer_name          0
pack_size_label            0
short_composition1         0
short_composition2    123120
Is_discontinued            0
id_y                       0
substitute0             8729
substitute1            13063
substitute2            16394
substitute3            19488
substitute4            22130
sideEffect0                0
sideEffect1             8855
sideEffect2            16775
sideEffect3            36076
sideEffect4            74486
use0                       0
use1                  155378
use2                  195922
use3                  214750
use4                  216876
Therapeutic Class         63
Action Class           98547
Quantity Available         0
dtype: int64


In [82]:
df_merged.fillna("Unknown", inplace=True)

In [83]:
# Save merged dataset as a CSV file in Kaggle's working directory
df_merged.to_csv("/kaggle/working/medicine_dataset_1.csv", index=False)

In [86]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [84]:
df = pd.read_csv("/kaggle/working/medicine_dataset_1.csv")

In [85]:
print(f" Total Medicines in Dataset: {len(df)}")

 Total Medicines in Dataset: 221415


In [87]:
df["combined_text"] = (
    df["name"] + " " +
    df["short_composition1"].fillna("") + " " +
    df["short_composition2"].fillna("") + " " +
    df["pack_size_label"].fillna("")
).str.lower()

In [88]:
#Generate embeddings
embeddings = model.encode(df["combined_text"].tolist(), convert_to_numpy=True, normalize_embeddings=True)
#faiss.normalize_L2(embeddings)

Batches:   0%|          | 0/6920 [00:00<?, ?it/s]

In [89]:
index = faiss.IndexFlatIP(embeddings.shape[1])  # Use IndexFlatIP for cosine similarity
index.add(np.array(embeddings))

In [90]:
# Save FAISS index
faiss.write_index(index, "/kaggle/working/medicine_faiss_4.index")

In [91]:
df.head()

,id_x,name,price(₹),manufacturer_name,pack_size_label,short_composition1,short_composition2,Is_discontinued,id_y,substitute0,...,sideEffect4,use0,use1,use2,use3,use4,Therapeutic Class,Action Class,Quantity Available,combined_text
0,1,augmentin 625 duo tablet,223.42,Glaxo SmithKline Pharmaceuticals Ltd,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg),False,1,Penciclav 500 mg/125 mg Tablet,...,Unknown,Treatment of Bacterial infections,Unknown,Unknown,Unknown,Unknown,ANTI INFECTIVES,Unknown,19,augmentin 625 duo tablet amoxycillin (500mg) ...
1,2,azithral 500 tablet,132.36,Alembic Pharmaceuticals Ltd,strip of 5 tablets,Azithromycin (500mg),Unknown,False,2,Zithrocare 500mg Tablet,...,Unknown,Treatment of Bacterial infections,Unknown,Unknown,Unknown,Unknown,ANTI INFECTIVES,Macrolides,0,azithral 500 tablet azithromycin (500mg) unkno...
2,3,ascoril ls syrup,118.00,Glenmark Pharmaceuticals Ltd,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml),False,3,Solvin LS Syrup,...,Stomach pain,Treatment of Cough with mucus,Unknown,Unknown,Unknown,Unknown,RESPIRATORY,Unknown,12,ascoril ls syrup ambroxol (30mg/5ml) levosal...
3,4,allegra 120mg tablet,218.81,Sanofi India Ltd,strip of 10 tablets,Fexofenadine (120mg),Unknown,False,4,Lcfex Tablet,...,Unknown,Treatment of Sneezing and runny nose due to al...,Treatment of Allergic conditions,Unknown,Unknown,Unknown,RESPIRATORY,H1 Antihistaminics (second Generation),6,allegra 120mg tablet fexofenadine (120mg) unkn...
4,5,avil 25 tablet,10.96,Sanofi India Ltd,strip of 15 tablets,Pheniramine (25mg),Unknown,False,5,Eralet 25mg Tablet,...,Unknown,Treatment of Allergic conditions,Unknown,Unknown,Unknown,Unknown,RESPIRATORY,H1 Antihistaminics (First Generation),12,avil 25 tablet pheniramine (25mg) unknown stri...


In [92]:
df.drop("id_y", axis = 1, inplace = True)

In [93]:
df.rename(columns={'id_x': 'id', 'price(₹)': 'price', 'Quantity Available': 'quantity_available', "Therapeutic Class": "therapeutic_class", "Action Class" : "action_class"}, inplace=True)

In [94]:
# Save updated dataset with embeddings
df.to_csv("/kaggle/working/processed_medicine_dataset_5.csv", index=False)

In [95]:
index = faiss.read_index("/kaggle/working/medicine_faiss_4.index")
df = pd.read_csv("/kaggle/working/processed_medicine_dataset_5.csv")

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221415 entries, 0 to 221414
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  221415 non-null  int64  
 1   name                221415 non-null  object 
 2   price               221415 non-null  float64
 3   manufacturer_name   221415 non-null  object 
 4   pack_size_label     221415 non-null  object 
 5   short_composition1  221415 non-null  object 
 6   short_composition2  221415 non-null  object 
 7   Is_discontinued     221415 non-null  bool   
 8   substitute0         221415 non-null  object 
 9   substitute1         221415 non-null  object 
 10  substitute2         221415 non-null  object 
 11  substitute3         221415 non-null  object 
 12  substitute4         221415 non-null  object 
 13  sideEffect0         221415 non-null  object 
 14  sideEffect1         221415 non-null  object 
 15  sideEffect2         221415 non-nul

In [98]:
def search_medicine(query_text, top_k=5):
    """
    Searches for the most relevant medicines using FAISS with cosine similarity.
    """
    query_embedding = model.encode([query_text], convert_to_numpy=True, normalize_embeddings=True)

    # ✅ Normalize embeddings for cosine similarity
    faiss.normalize_L2(query_embedding)
    faiss.normalize_L2(embeddings)

    distances, indices = index.search(query_embedding, k=top_k)

    matched_medicines = []
    for i, idx in enumerate(indices[0]):
        medicine = df.iloc[idx][["name", "manufacturer_name", "pack_size_label",
                                 "short_composition1", "short_composition2",
                                 "substitute0", "substitute1", "substitute2",
                                 "substitute3", "substitute4"]].to_dict()
        medicine["similarity_score"] = float(1 - distances[0][i])  # Convert L2 to similarity
        matched_medicines.append(medicine)

    return sorted(matched_medicines, key=lambda x: x["similarity_score"], reverse=True)

In [99]:
# Example usage
query = "Amoxicllin"
matches = search_medicine(query)
print("Best Matches (Ranked by Similarity):")
matches

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Best Matches (Ranked by Similarity):


[{'name': 'amoxysain 250mg capsule',
  'manufacturer_name': 'Sain Medicaments Pvt Ltd',
  'pack_size_label': 'strip of 10 capsules',
  'short_composition1': 'Amoxycillin (250mg)',
  'short_composition2': 'Unknown',
  'substitute0': 'Amoxipen 250mg Capsule',
  'substitute1': 'Hipen A 250mg Capsule',
  'substitute2': 'Moxipal 250mg Capsule',
  'substitute3': 'Amoxil 250mg Capsule',
  'substitute4': 'Moxikem 250mg Capsule',
  'similarity_score': 0.3273969292640686},
 {'name': 'afymox-cl tablet',
  'manufacturer_name': 'Sanify Healthcare Pvt  Ltd',
  'pack_size_label': 'strip of 10 tablets',
  'short_composition1': 'Amoxycillin (125mg) ',
  'short_composition2': ' Cloxacillin (125mg) ',
  'substitute0': 'Unknown',
  'substitute1': 'Unknown',
  'substitute2': 'Unknown',
  'substitute3': 'Unknown',
  'substitute4': 'Unknown',
  'similarity_score': 0.32341957092285156},
 {'name': 'amoclox tablet',
  'manufacturer_name': 'Bhavishya Pharmaceuticals Pvt Ltd',
  'pack_size_label': 'strip of 10 ta

In [238]:
import re
import pandas as pd
from rapidfuzz import process, fuzz

def clean_text(text):
    """
    Cleans input text by keeping alphanumeric characters and converting to lowercase.
    Preserves spaces and numbers for better matching.
    """
    if pd.isna(text) or not isinstance(text, str):  # Handle NaN and non-string values
        return ""
    return re.sub(r'[^a-zA-Z0-9 ]', '', text).lower().strip()  # Keep letters, numbers, spaces

def fuzzy_medicine_search(query, df, threshold=50, top_k=10):
    """
    Searches for medicines in the DataFrame using fuzzy matching (RapidFuzz).
    """
    # Step 1: Clean the query text
    query_clean = clean_text(query)
    if len(query_clean) < 2:  # Prevent invalid queries
        return None

    # Step 2: Apply Fuzzy Matching on Full Medicine List
    medicine_names = df["name"].dropna().tolist()
    fuzzy_results = process.extract(query_clean, medicine_names, scorer=fuzz.WRatio, limit=top_k)

    # Step 3: Format Results
    matched_medicines = []
    for match, score, _ in fuzzy_results:
        if score >= threshold:
            row = df[df["name"] == match].iloc[0]  # Get the first matching row
            matched_medicines.append({
                "name": row["name"],
                "manufacturer_name": row.get("manufacturer_name", "Unknown"),
                "pack_size_label": row.get("pack_size_label", "Unknown"),
                "similarity_score": score / 100.0  # Normalize score
            })

    return matched_medicines if matched_medicines else None

In [239]:
query_text = "Amoxicillin"
results = fuzzy_medicine_search(query_text, df, top_k=5)

if results:
    results_df_1 = pd.DataFrame(results)
else:
    print("No match found")

In [240]:
results_df_1

,name,manufacturer_name,pack_size_label,similarity_score
0,amoxycillin 500mg capsule,Jagsonpal Pharmaceuticals Ltd,strip of 10 capsules,0.818182
1,amoxycillin 125mg oral suspension,Nestor Pharmaceuticals,bottle of 30 ml Oral Suspension,0.818182
2,amoxycillin 250mg capsule,Torque Pharmaceuticals Pvt Ltd,strip of 10 capsules,0.818182
3,amoxycillin 125mg syrup,Lark Laboratories Ltd,bottle of 60 ml Syrup,0.818182
4,amoxycillin 125mg tablet,Makers Laboratories Ltd,strip of 10 tablets,0.818182


In [249]:
query_text = "Paracitamol"
results = fuzzy_medicine_search(query_text, df, top_k=5)

if results:
    results_df_2 = pd.DataFrame(results)
else:
    print("No match found")

In [250]:
results_df_2

,name,manufacturer_name,pack_size_label,similarity_score
0,dr best paracetamol 250 oral suspension,DR Best pharmaceuticals Pvt Ltd,bottle of 60 ml Oral Suspension,0.818182
1,davaindia nimesulide+paracetamol 100mg/325mg t...,Davaindia Generic Pharmacy,strip of 10 tablets,0.818182
2,davaindia diclofenac+paracetamol 50mg/325mg ta...,Davaindia Generic Pharmacy,strip of 10 tablets,0.818182
3,davaindia diclofenac+paracetamol+serratiopepti...,Davaindia Generic Pharmacy,strip of 10 tablets,0.818182
4,davaindia paracetamol 500mg tablet,Davaindia Generic Pharmacy,strip of 10 tablets,0.818182


In [245]:
import pandas as pd
from rapidfuzz import process, fuzz

def evaluate_fuzzy_search(df, test_samples, threshold=50, top_k=10):
    """
    Evaluates fuzzy search by comparing predicted names to actual correct names.
    Metrics: Mean Similarity Score, Top-K Match Rate.
    """
    total_similarity = 0
    correct_predictions = 0
    total_samples = len(test_samples)

    for query, actual in test_samples:
        fuzzy_results = process.extract(query, df["name"].dropna().tolist(), scorer=fuzz.WRatio, limit=top_k)

        # Check if actual medicine appears in top-k results
        found = False
        for match, score, _ in fuzzy_results:
            if actual.lower() == match.lower():  # Correct prediction
                total_similarity += score
                correct_predictions += 1
                found = True
                break

        # If not found, add a penalty (optional)
        if not found:
            total_similarity += 0  # You can assign a low similarity score if no correct match found

    # Compute metrics
    mean_similarity = total_similarity / total_samples
    top_k_match_rate = correct_predictions / total_samples  # % of times the correct match appears in top-k

    return {
        "Mean Similarity Score": round(mean_similarity, 2),
        "Top-K Match Rate": round(top_k_match_rate, 2)
    }

# Example dataset (Test cases)
test_samples = [
    ("Amocicylin", "Amoxicillin 500mg Tablet"),  # Misspelled query
    ("Azitromicin", "Azithromycin 500mg Tablet"),
    ("Paraetamol", "Paracetamol 650mg Tablet"),
    ("Ibrufen", "Ibuprofen 200mg Tablet"),
    ("Allegra 120mg", "Allegra 120mg Tablet")
]


# Evaluate the function
accuracy_results = evaluate_fuzzy_search(df, test_samples, top_k=10)
print("\n Accuracy Evaluation Metrics:")
print(accuracy_results)



 Accuracy Evaluation Metrics:
{'Mean Similarity Score': 17.1, 'Top-K Match Rate': 0.2}


In [254]:
from rapidfuzz.distance import Levenshtein

def compute_levenshtein(true_name, predicted_name):
    """
    Computes the normalized Levenshtein similarity score between two strings.
    Returns a score between 0 and 1 (higher is better).
    """
    true_name = true_name.lower().strip()
    predicted_name = predicted_name.lower().strip()

    # Compute Levenshtein distance
    lev_distance = Levenshtein.distance(true_name, predicted_name)
    
    # Normalize to similarity score (1 - normalized distance)
    max_len = max(len(true_name), len(predicted_name))
    similarity_score = 1 - (lev_distance / max_len)

    return round(similarity_score, 4)  # Return similarity between 0 and 1

In [259]:
score = compute_levenshtein("Amoxicillin", "Amoxicylin")
print(f"Levenshtein Similarity: {score}")

Levenshtein Similarity: 0.8182
